## 模擬每日指數報酬率

In [1]:
import pandas as pd
from openpyxl import load_workbook
import openpyxl

### Step1. `取得昨天的收盤資料以及市值資料`

In [2]:
# 數據是昨天算好的
Factor_Information = pd.read_excel('Base_Data.xlsx')
Factor_Information.head(3)

,年月日,證券代碼,流通在外股數,收盤價(元)
0,2025-04-24,2357 華碩,742760.280,544.0
1,2025-04-24,2345 智邦,561117.897,516.0
2,2025-04-24,6669 緯穎,185840.791,1865.0


In [3]:
# 重新命名欄位名稱，避免程式執行錯誤
Factor_Information['年月日'] = pd.to_datetime(Factor_Information['年月日'], format='%Y-%m-%d')
yesterdaytime = Factor_Information['年月日'].dt.strftime('%Y-%m-%d')[0]
Factor_Information

,年月日,證券代碼,流通在外股數,收盤價(元)
0,2025-04-24,2357 華碩,742760.280,544.00
1,2025-04-24,2345 智邦,561117.897,516.00
2,2025-04-24,6669 緯穎,185840.791,1865.00
3,2025-04-24,3008 大立光,133468.197,2140.00
4,2025-04-24,2207 和泰車,557102.768,606.00
...,...,...,...,...
397,2025-04-24,8374 羅昇,112250.487,80.10
398,2025-04-24,1532 勤美,403912.097,25.35
399,2025-04-24,2617 台航,417294.487,27.50
400,2025-04-24,1810 和成,302303.719,17.35


In [4]:
Factor_Information.columns = ['Date','Company','Outstanding_Share',f'{yesterdaytime}_Stock_Price']

Factor_Information = Factor_Information.drop(columns=['Date'])

# 確保資料執行的型態
Factor_Information['Company'] = Factor_Information['Company'].astype(str)

# 計算市值
Factor_Information[f'{yesterdaytime}_Factor_MV'] = (Factor_Information['Outstanding_Share'] * Factor_Information[f'{yesterdaytime}_Stock_Price'])

# 計算權重
Total_MV = Factor_Information[f'{yesterdaytime}_Factor_MV'].sum()
Factor_Information['Factor_Weight'] = Factor_Information[f'{yesterdaytime}_Factor_MV'] / Total_MV

# 拆分column，把Factor拆成Factor 跟 Chinese_name
Factor_Information['Factor'] = Factor_Information['Company'].str[:4]
Factor_Information['Chinese_name'] = Factor_Information['Company'].str[4:]
Factor_Information = Factor_Information.drop(columns=['Company'])

#重新排序column順序
Factor_Information = Factor_Information[['Factor','Chinese_name','Outstanding_Share',f'{yesterdaytime}_Stock_Price',f'{yesterdaytime}_Factor_MV','Factor_Weight']]

Factor_Information

,Factor,Chinese_name,Outstanding_Share,2025-04-24_Stock_Price,2025-04-24_Factor_MV,Factor_Weight
0,2357,華碩,742760.280,544.00,4.040616e+08,0.026172
1,2345,智邦,561117.897,516.00,2.895368e+08,0.018754
2,6669,緯穎,185840.791,1865.00,3.465931e+08,0.022449
3,3008,大立光,133468.197,2140.00,2.856219e+08,0.018500
4,2207,和泰車,557102.768,606.00,3.376043e+08,0.021867
...,...,...,...,...,...,...
397,8374,羅昇,112250.487,80.10,8.991264e+06,0.000582
398,1532,勤美,403912.097,25.35,1.023917e+07,0.000663
399,2617,台航,417294.487,27.50,1.147560e+07,0.000743
400,1810,和成,302303.719,17.35,5.244970e+06,0.000340


### Step2. `下載今天的未調整收盤價以及計算報酬率`
> 資料來源是Tej，要每天更新

In [5]:
All_Stock_Today_Close_Price = pd.read_excel('Tej_close_price(update_everyday).xlsx',sheet_name='close_price')
All_Stock_Today_Close_Price

,證券代碼,年月日,收盤價(元)
0,1101 台泥,2025-04-25,29.35
1,1102 亞泥,2025-04-25,43.95
2,1103 嘉泥,2025-04-25,14.95
3,1104 環泥,2025-04-25,27.60
4,1108 幸福,2025-04-25,13.75
...,...,...,...
1853,9951 皇田,2025-04-25,51.40
1854,9955 佳龍,2025-04-25,33.65
1855,9958 世紀鋼,2025-04-25,173.50
1856,9960 邁達康,2025-04-25,24.60


In [6]:
# 稍微調整一下欄位，方便後續資料處理
All_Stock_Today_Close_Price['Factor'] = All_Stock_Today_Close_Price['證券代碼'].str[:4]
All_Stock_Today_Close_Price['Chinese_name'] = All_Stock_Today_Close_Price['證券代碼'].str[5:]
All_Stock_Today_Close_Price = All_Stock_Today_Close_Price.drop(columns=['證券代碼'])
All_Stock_Today_Close_Price

,年月日,收盤價(元),Factor,Chinese_name
0,2025-04-25,29.35,1101,台泥
1,2025-04-25,43.95,1102,亞泥
2,2025-04-25,14.95,1103,嘉泥
3,2025-04-25,27.60,1104,環泥
4,2025-04-25,13.75,1108,幸福
...,...,...,...,...
1853,2025-04-25,51.40,9951,皇田
1854,2025-04-25,33.65,9955,佳龍
1855,2025-04-25,173.50,9958,世紀鋼
1856,2025-04-25,24.60,9960,邁達康


In [7]:
# 重新命名，要記得加上日期，才不會搞錯
All_Stock_Today_Close_Price['年月日'] = pd.to_datetime(All_Stock_Today_Close_Price['年月日'], format='%Y-%m-%d')
todaytime = All_Stock_Today_Close_Price['年月日'].dt.strftime('%Y%m%d')[0]
All_Stock_Today_Close_Price.columns = ['Date',f'{todaytime}_Stock_Price','Factor','Chinese_name']
All_Stock_Today_Close_Price = All_Stock_Today_Close_Price.drop(columns=['Chinese_name'])

#重新排序column順序
All_Stock_Today_Close_Price = All_Stock_Today_Close_Price[['Factor','Date',f'{todaytime}_Stock_Price']]

All_Stock_Today_Close_Price

,Factor,Date,20250425_Stock_Price
0,1101,2025-04-25,29.35
1,1102,2025-04-25,43.95
2,1103,2025-04-25,14.95
3,1104,2025-04-25,27.60
4,1108,2025-04-25,13.75
...,...,...,...
1853,9951,2025-04-25,51.40
1854,9955,2025-04-25,33.65
1855,9958,2025-04-25,173.50
1856,9960,2025-04-25,24.60


> 合併需要的欄位

In [8]:
Factor_Information['Factor'] = pd.to_numeric(Factor_Information['Factor'],errors='coerce')
All_Stock_Today_Close_Price['Factor'] = pd.to_numeric(All_Stock_Today_Close_Price['Factor'],errors='coerce')

In [9]:
History_Data_df = pd.merge(Factor_Information, All_Stock_Today_Close_Price, on='Factor', how='left')
History_Data_df

,Factor,Chinese_name,Outstanding_Share,2025-04-24_Stock_Price,2025-04-24_Factor_MV,Factor_Weight,Date,20250425_Stock_Price
0,2357,華碩,742760.280,544.00,4.040616e+08,0.026172,2025-04-25,591.00
1,2345,智邦,561117.897,516.00,2.895368e+08,0.018754,2025-04-25,552.00
2,6669,緯穎,185840.791,1865.00,3.465931e+08,0.022449,2025-04-25,1905.00
3,3008,大立光,133468.197,2140.00,2.856219e+08,0.018500,2025-04-25,2225.00
4,2207,和泰車,557102.768,606.00,3.376043e+08,0.021867,2025-04-25,612.00
...,...,...,...,...,...,...,...,...
397,8374,羅昇,112250.487,80.10,8.991264e+06,0.000582,2025-04-25,81.70
398,1532,勤美,403912.097,25.35,1.023917e+07,0.000663,2025-04-25,25.65
399,2617,台航,417294.487,27.50,1.147560e+07,0.000743,2025-04-25,27.65
400,1810,和成,302303.719,17.35,5.244970e+06,0.000340,2025-04-25,17.30


In [10]:
# 計算我們需要的價格變動（回報），以獲得新成份股的每日回報
History_Data_df[f'{todaytime}_Net_Return'] = (History_Data_df[f'{todaytime}_Stock_Price'] / History_Data_df[f'{yesterdaytime}_Stock_Price']) - 1
History_Data_df['Percentage_of_each_stock_return'] = History_Data_df['Factor_Weight'] * History_Data_df[f'{todaytime}_Net_Return']

# 重新排列 DataFrame 的欄位
new_column_order = ['Date','Factor','Chinese_name','Outstanding_Share',f'{yesterdaytime}_Factor_MV',f'{yesterdaytime}_Stock_Price',f'{todaytime}_Stock_Price','Factor_Weight',f'{todaytime}_Net_Return','Percentage_of_each_stock_return']
History_Data_df = History_Data_df[new_column_order]

History_Data_df

,Date,Factor,Chinese_name,Outstanding_Share,2025-04-24_Factor_MV,2025-04-24_Stock_Price,20250425_Stock_Price,Factor_Weight,20250425_Net_Return,Percentage_of_each_stock_return
0,2025-04-25,2357,華碩,742760.280,4.040616e+08,544.00,591.00,0.026172,0.086397,2.261145e-03
1,2025-04-25,2345,智邦,561117.897,2.895368e+08,516.00,552.00,0.018754,0.069767,1.308394e-03
2,2025-04-25,6669,緯穎,185840.791,3.465931e+08,1865.00,1905.00,0.022449,0.021448,4.814851e-04
3,2025-04-25,3008,大立光,133468.197,2.856219e+08,2140.00,2225.00,0.018500,0.039720,7.348159e-04
4,2025-04-25,2207,和泰車,557102.768,3.376043e+08,606.00,612.00,0.021867,0.009901,2.165052e-04
...,...,...,...,...,...,...,...,...,...,...
397,2025-04-25,8374,羅昇,112250.487,8.991264e+06,80.10,81.70,0.000582,0.019975,1.163295e-05
398,2025-04-25,1532,勤美,403912.097,1.023917e+07,25.35,25.65,0.000663,0.011834,7.848559e-06
399,2025-04-25,2617,台航,417294.487,1.147560e+07,27.50,27.65,0.000743,0.005455,4.054298e-06
400,2025-04-25,1810,和成,302303.719,5.244970e+06,17.35,17.30,0.000340,-0.002882,-9.790284e-07


### Step3. `儲存為 Excel 檔案，方便保存記錄.`

In [11]:
# 備份檔案
History_Data_df.to_excel(f'{todaytime}_資料紀錄.xlsx',index=False)

### Step4. `使用今日的資料將其重組為基本資訊 (供明日使用)`

In [13]:
new_columns = ['Date','Factor','Chinese_name','Outstanding_Share',f'{todaytime}_Stock_Price']

Historical_Record_df = History_Data_df[new_columns]
Historical_Record_df['Factor'] = Historical_Record_df['Factor'].astype(str)
Historical_Record_df['證券代碼'] = Historical_Record_df['Factor'] + Historical_Record_df['Chinese_name'] 

Historical_Record_df = Historical_Record_df.drop(columns=['Factor'])
Historical_Record_df = Historical_Record_df.drop(columns=['Chinese_name'])

Historical_Record_df.columns = ['年月日','流通在外股數','收盤價(元)','證券代碼']

New_order = ['年月日','證券代碼','流通在外股數','收盤價(元)']
Historical_Record_df = Historical_Record_df[New_order]
Historical_Record_df

C:\Users\joshchiang\AppData\Local\Temp\ipykernel_11728\1447464745.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Historical_Record_df['Factor'] = Historical_Record_df['Factor'].astype(str)
C:\Users\joshchiang\AppData\Local\Temp\ipykernel_11728\1447464745.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Historical_Record_df['證券代碼'] = Historical_Record_df['Factor'] + Historical_Record_df['Chinese_name']


,年月日,證券代碼,流通在外股數,收盤價(元)
0,2025-04-25,2357 華碩,742760.280,591.00
1,2025-04-25,2345 智邦,561117.897,552.00
2,2025-04-25,6669 緯穎,185840.791,1905.00
3,2025-04-25,3008 大立光,133468.197,2225.00
4,2025-04-25,2207 和泰車,557102.768,612.00
...,...,...,...,...
397,2025-04-25,8374 羅昇,112250.487,81.70
398,2025-04-25,1532 勤美,403912.097,25.65
399,2025-04-25,2617 台航,417294.487,27.65
400,2025-04-25,1810 和成,302303.719,17.30


In [14]:
Historical_Record_df.to_excel('Base_Data.xlsx',index=False)

### Step5. `將模擬的每日報表儲存在指定檔案的特定欄位中，以確保代碼可重複使用。`

In [16]:
Today_Index_Net_Return = History_Data_df['Percentage_of_each_stock_return'].sum()
Today_Index_Net_Return

0.02439925202017114

In [ ]:
excel_file = 'Tej_close_price(update_everyday).xlsx'
sheet_name = 'index_return'
column_letter = 'H'  

wb = openpyxl.load_workbook(excel_file)

sheet = wb[sheet_name]

row = 3
while sheet[f"{column_letter}{row}"].value is not None:
    row += 1

sheet[f"{column_letter}{row}"] = Today_Index_Net_Return

wb.save(excel_file)

wb.close()